In [1]:
"""
===============================================================================
DENOISING SCRIPT - CS506 Project
===============================================================================
Complete with all visualization functions
===============================================================================
"""

import os
import shutil
import cv2
import numpy as np
import random
import sys
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Force output flushing
sys.stdout.flush()

# Set seed for reproducibility
random.seed("This is a seed.")
np.random.seed(42)

# ============ PATH CONFIGURATION ============
# Clean originals (used for PSNR / SSIM comparisons)
BASE_PATH = "../../Photos_Subset/Original"  

# Input paths for noised images
BW_INPUT = "../../Photos_Subset/Noised_Images/BW" 
COLOUR_INPUT = "../../Photos_Subset/Noised_Images/Color" 

# Output paths
OUTPUT_SP = "../../Data_Results/Data/finished/salt_pepper"
OUTPUT_SPECKLE = "../../Data_Results/Data/finished/speckle"

# Clean output directories if they exist
for output_dir in [OUTPUT_SP, OUTPUT_SPECKLE]:
    if os.path.exists(output_dir):
        print(f"Cleaning existing output directory: {output_dir}")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

print(f"✓ Base (clean) images: {BASE_PATH}")
print(f"✓ BW noised images: {BW_INPUT}")
print(f"✓ Colour noised images: {COLOUR_INPUT}")
print(f"✓ Salt & Pepper output: {OUTPUT_SP}")
print(f"✓ Speckle output: {OUTPUT_SPECKLE}\n")

# ======================== UTILS ========================
def _resize_to_match(ref, img):
    """Resize img to have the same H×W as ref (channels preserved)."""
    if img.shape[:2] != ref.shape[:2]:
        img = cv2.resize(img, (ref.shape[1], ref.shape[0]), interpolation=cv2.INTER_CUBIC)
    return img

# ======================== SPECKLE: LOG-DOMAIN NLM ========================
def denoise_speckle_nlm(image, h=20.0, templateWindowSize=7, searchWindowSize=21):
    """Speckle denoising using log-domain Non-Local Means (NLM)."""
    eps = 1e-3
    
    # GRAYSCALE CASE
    if image.ndim == 2:
        Y = image.astype(np.float32) / 255.0
        Y_log = np.log(Y + eps)
        log_min, log_max = Y_log.min(), Y_log.max()
        log_range = log_max - log_min + 1e-8
        Y_log_norm = (Y_log - log_min) / log_range
        Y_log_u8 = (Y_log_norm * 255.0).astype(np.uint8)
        Y_log_d = cv2.fastNlMeansDenoising(
            Y_log_u8, None, h=h, templateWindowSize=templateWindowSize, 
            searchWindowSize=searchWindowSize
        )
        Y_log_d = Y_log_d.astype(np.float32) / 255.0
        Y_log_d = Y_log_d * log_range + log_min
        Y_d = np.exp(Y_log_d) - eps
        Y_d = np.clip(Y_d * 255.0, 0, 255).astype(np.uint8)
        return Y_d
    
    # COLOR CASE
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(ycrcb)
    Yf = Y.astype(np.float32) / 255.0
    Y_log = np.log(Yf + eps)
    log_min, log_max = Y_log.min(), Y_log.max()
    log_range = log_max - log_min + 1e-8
    Y_log_norm = (Y_log - log_min) / log_range
    Y_log_u8 = (Y_log_norm * 255.0).astype(np.uint8)
    Y_log_d = cv2.fastNlMeansDenoising(
        Y_log_u8, None, h=h, templateWindowSize=templateWindowSize,
        searchWindowSize=searchWindowSize
    )
    Y_log_d = Y_log_d.astype(np.float32) / 255.0
    Y_log_d = Y_log_d * log_range + log_min
    Y_d = np.exp(Y_log_d) - eps
    Y_d = np.clip(Y_d * 255.0, 0, 255).astype(np.uint8)
    ycrcb_d = cv2.merge([Y_d, Cr, Cb])
    return cv2.cvtColor(ycrcb_d, cv2.COLOR_YCrCb2BGR)

# ======================== SALT & PEPPER: K-MEDOIDS ========================
def denoise_salt_pepper_kmedoids(image, n_clusters=3, sample_fraction=0.05, max_iter=30, use_median_prefilter=True):
    """Denoise salt & pepper using K-medoids clustering."""
    original_shape = image.shape
    
    if use_median_prefilter:
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        salt_mask = gray > 240
        pepper_mask = gray < 15
        noise_mask = salt_mask | pepper_mask
        
        median_filtered = cv2.medianBlur(image, 5)
        working_image = image.copy()
        working_image[noise_mask] = median_filtered[noise_mask]
    else:
        working_image = image.copy()
    
    if len(working_image.shape) == 3:
        pixels = working_image.reshape(-1, 3).astype(np.float64)
    else:
        pixels = working_image.reshape(-1, 1).astype(np.float64)
    
    n_total_pixels = len(pixels)
    sample_size = max(int(n_total_pixels * sample_fraction), n_clusters * 50)
    sample_size = min(sample_size, n_total_pixels)
    
    sample_indices = np.random.choice(n_total_pixels, sample_size, replace=False)
    sample_pixels = pixels[sample_indices]
    medoid_indices = np.random.choice(len(sample_pixels), n_clusters, replace=False)
    
    for _ in range(max_iter):
        distances = np.zeros((len(sample_pixels), n_clusters))
        for i, medoid_idx in enumerate(medoid_indices):
            distances[:, i] = np.linalg.norm(sample_pixels - sample_pixels[medoid_idx], axis=1)
        
        labels = np.argmin(distances, axis=1)
        new_medoid_indices = np.zeros(n_clusters, dtype=int)
        
        for cluster_id in range(n_clusters):
            cluster_mask = (labels == cluster_id)
            if np.sum(cluster_mask) == 0:
                new_medoid_indices[cluster_id] = medoid_indices[cluster_id]
                continue
            
            cluster_point_indices = np.where(cluster_mask)[0]
            cluster_points = sample_pixels[cluster_mask]
            cluster_mean = np.mean(cluster_points, axis=0)
            distances_to_mean = np.linalg.norm(cluster_points - cluster_mean, axis=1)
            best_point_in_cluster = cluster_point_indices[np.argmin(distances_to_mean)]
            new_medoid_indices[cluster_id] = best_point_in_cluster
        
        if np.array_equal(medoid_indices, new_medoid_indices):
            break
        
        medoid_indices = new_medoid_indices
    
    final_medoids = sample_pixels[medoid_indices]
    distances_all = np.zeros((n_total_pixels, n_clusters))
    for i in range(n_clusters):
        distances_all[:, i] = np.linalg.norm(pixels - final_medoids[i], axis=1)
    
    labels_all = np.argmin(distances_all, axis=1)
    denoised_pixels = final_medoids[labels_all]
    denoised = denoised_pixels.reshape(original_shape)
    
    return np.clip(denoised, 0, 255).astype(np.uint8)

# ======================== METRIC CALCULATION ========================
def calculate_entropy(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    hist, _ = np.histogram(gray.flatten(), bins=256, range=(0, 256))
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    return entropy(hist, base=2)

def calculate_sharpness(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    return laplacian.var()

def calculate_spatial_frequency(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    gray = gray.astype(np.float64)
    RF = np.sqrt(np.mean(np.diff(gray, axis=1) ** 2))
    CF = np.sqrt(np.mean(np.diff(gray, axis=0) ** 2))
    SF = np.sqrt(RF ** 2 + CF ** 2)
    return SF

def calculate_dynamic_range(image):
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    return np.max(gray) - np.min(gray)

def calculate_noise_variance(original, noisy):
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        noisy_gray = cv2.cvtColor(noisy, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        noisy_gray = noisy
    noise = noisy_gray.astype(np.float64) - original_gray.astype(np.float64)
    return np.var(noise)

def calculate_all_metrics(original, denoised, noisy):
    if len(original.shape) == 3:
        original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
        denoised_gray = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original
        denoised_gray = denoised
    
    psnr_value = psnr(original_gray, denoised_gray, data_range=255)
    ssim_value = ssim(original_gray, denoised_gray, data_range=255)
    mse_value = np.mean((original_gray.astype(np.float64) - denoised_gray.astype(np.float64)) ** 2)
    
    entropy_orig = calculate_entropy(original)
    entropy_denoised = calculate_entropy(denoised)
    entropy_diff = abs(entropy_denoised - entropy_orig)
    
    noise_var = calculate_noise_variance(original, denoised)
    sharpness = calculate_sharpness(denoised)
    spatial_freq = calculate_spatial_frequency(denoised)
    dynamic_range = calculate_dynamic_range(denoised)
    
    return {
        "PSNR": psnr_value,
        "SSIM": ssim_value,
        "MSE": mse_value,
        "Entropy_Diff": entropy_diff,
        "Noise_Variance": noise_var,
        "Sharpness": sharpness,
        "Spatial_Freq": spatial_freq,
        "Dynamic_Range": dynamic_range
    }

# ======================== DENOISING PARAMETERS ========================
SP_DENOISE_PARAMS = {
    "kmedoids": [
        {"n_clusters": 3, "use_median_prefilter": False, "name": "kmedoids_k3_no_prefilter"},
        {"n_clusters": 5, "use_median_prefilter": False, "name": "kmedoids_k5_no_prefilter"},
        {"n_clusters": 8, "use_median_prefilter": False, "name": "kmedoids_k8_no_prefilter"},
        {"n_clusters": 3, "use_median_prefilter": True, "name": "kmedoids_k3_median"},
        {"n_clusters": 5, "use_median_prefilter": True, "name": "kmedoids_k5_median"},
        {"n_clusters": 8, "use_median_prefilter": True, "name": "kmedoids_k8_median"},
        {"n_clusters": 10, "use_median_prefilter": True, "name": "kmedoids_k10_median"},
        {"n_clusters": 12, "use_median_prefilter": True, "name": "kmedoids_k12_median"},
    ]
}

SPECKLE_DENOISE_PARAMS = {
    "nlm": [
        {"h": 14.0, "template": 7, "search": 21, "name": "nlm_h14_t7_s21"},
        {"h": 17.0, "template": 7, "search": 21, "name": "nlm_h17_t7_s21"},
        {"h": 20.0, "template": 7, "search": 21, "name": "nlm_h20_t7_s21"},
        {"h": 23.0, "template": 7, "search": 21, "name": "nlm_h23_t7_s21"},
        {"h": 20.0, "template": 9, "search": 31, "name": "nlm_h20_t9_s31"},
        {"h": 23.0, "template": 9, "search": 31, "name": "nlm_h23_t9_s31"},
    ]
}

# ======================== BEST PARAMETER SELECTION ========================
def select_best_parameter_psnr_detail(all_results, psnr_tolerance=0.5):
    """Choose best parameter with highest PSNR, then Sharpness, then SSIM."""
    best_param = None
    best_psnr = -np.inf
    best_sharp = -np.inf
    best_ssim = -np.inf
    
    for param_name, df in all_results.items():
        m_psnr = df["PSNR"].mean()
        m_sharp = df["Sharpness"].mean()
        m_ssim = df["SSIM"].mean()
        
        if m_psnr > best_psnr + psnr_tolerance:
            best_param = param_name
            best_psnr = m_psnr
            best_sharp = m_sharp
            best_ssim = m_ssim
        elif best_psnr - m_psnr <= psnr_tolerance:
            if (m_sharp > best_sharp + 1e-6) or \
               (abs(m_sharp - best_sharp) <= 1e-6 and m_ssim > best_ssim):
                best_param = param_name
                best_psnr = m_psnr
                best_sharp = m_sharp
                best_ssim = m_ssim
    
    return best_param, best_psnr, best_ssim

# ======================== VISUALIZATION FUNCTIONS ========================

def create_comparison_graphs(all_results, graph_dir, noise_folder_name, best_param):
    """Create comparison graphs for all metrics"""
    print(f"    Creating graphs for {noise_folder_name}...")
    sys.stdout.flush()

    sns.set_style("whitegrid")
    metrics = ["PSNR", "SSIM", "MSE", "Entropy_Diff", "Noise_Variance",
               "Sharpness", "Spatial_Freq", "Dynamic_Range"]

    for metric in metrics:
        try:
            plt.figure(figsize=(12, 6))
            data_for_plot = []
            labels = []

            for param_name, df in all_results.items():
                data_for_plot.append(df[metric].values)
                labels.append(param_name)

            bp = plt.boxplot(data_for_plot, tick_labels=labels, patch_artist=True)

            for patch, label in zip(bp['boxes'], labels):
                if label == best_param:
                    patch.set_facecolor('lightgreen')
                    patch.set_edgecolor('darkgreen')
                    patch.set_linewidth(2)
                else:
                    patch.set_facecolor('lightblue')

            plt.title(f'{metric} Comparison - {noise_folder_name}', fontsize=14, fontweight='bold')
            plt.xlabel('Denoising Parameters', fontsize=12)
            plt.ylabel(metric, fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(graph_dir, f'{metric}.png'), dpi=300, bbox_inches='tight')
            plt.close()
        except Exception as e:
            print(f"      Error creating {metric} graph: {e}")

    # Best PSNR comparison
    try:
        fig, ax = plt.subplots(figsize=(14, 8))
        comparison_data = []
        for param_name, df in all_results.items():
            comparison_data.append({
                'Method': param_name,
                'PSNR': df['PSNR'].mean(),
                'SSIM': df['SSIM'].mean(),
                'MSE': df['MSE'].mean()
            })

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df = comparison_df.sort_values('PSNR', ascending=False)

        x = np.arange(len(comparison_df))
        width = 0.25

        bars1 = ax.bar(x - width, comparison_df['PSNR'], width, label='PSNR', color='steelblue')
        bars2 = ax.bar(x, comparison_df['SSIM'] * 50, width, label='SSIM (×50)', color='orange')
        bars3 = ax.bar(x + width, comparison_df['MSE'] / 100, width, label='MSE (÷100)', color='green')

        bars1[0].set_color('darkgreen')
        bars1[0].set_edgecolor('black')
        bars1[0].set_linewidth(2)

        ax.set_xlabel('Denoising Method', fontsize=12)
        ax.set_ylabel('Metric Value', fontsize=12)
        ax.set_title(f'Best PSNR Comparison - {noise_folder_name}', fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(comparison_df['Method'], rotation=45, ha='right')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)

        plt.tight_layout()
        plt.savefig(os.path.join(graph_dir, 'Best_PSNR_comparison.png'), dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"      Error creating Best PSNR comparison: {e}")

    # Heatmap
    try:
        fig, ax = plt.subplots(figsize=(14, 10))
        heatmap_data = []
        row_labels = []

        for param_name, df in all_results.items():
            row_data = [df[m].mean() for m in metrics]
            heatmap_data.append(row_data)
            row_labels.append(param_name)

        heatmap_array = np.array(heatmap_data)
        normalized = (heatmap_array - heatmap_array.min(axis=0)) / (heatmap_array.max(axis=0) - heatmap_array.min(axis=0) + 1e-10)

        im = ax.imshow(normalized, cmap='RdYlGn', aspect='auto')

        ax.set_xticks(np.arange(len(metrics)))
        ax.set_yticks(np.arange(len(row_labels)))
        ax.set_xticklabels(metrics, rotation=45, ha='right')
        ax.set_yticklabels(row_labels)

        for i in range(len(row_labels)):
            for j in range(len(metrics)):
                text = ax.text(j, i, f'{heatmap_array[i, j]:.2f}',
                              ha="center", va="center", color="black", fontsize=8)

        ax.set_title(f'all_metrics_individual - {noise_folder_name}', fontsize=14, fontweight='bold')
        fig.colorbar(im, ax=ax, label='Normalized Value')
        plt.tight_layout()
        plt.savefig(os.path.join(graph_dir, 'all_metrics_individual.png'), dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"      Error creating heatmap: {e}")

    # Comparison grid
    try:
        fig, axes = plt.subplots(3, 3, figsize=(18, 15))
        axes = axes.flatten()

        for idx, metric in enumerate(metrics):
            ax = axes[idx]
            data_for_plot = []
            labels = []

            for param_name, df in all_results.items():
                data_for_plot.append(df[metric].values)
                labels.append(param_name)

            bp = ax.boxplot(data_for_plot, tick_labels=labels, patch_artist=True)

            for patch, label in zip(bp['boxes'], labels):
                if label == best_param:
                    patch.set_facecolor('lightgreen')
                else:
                    patch.set_facecolor('lightblue')

            ax.set_title(metric, fontsize=11, fontweight='bold')
            ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
            ax.grid(axis='y', alpha=0.3)

        if len(metrics) < 9:
            axes[8].axis('off')

        plt.suptitle(f'comparison_grid - {noise_folder_name}\n(Green = Best: {best_param})',
                     fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(os.path.join(graph_dir, 'comparison_grid.png'), dpi=300, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(f"      Error creating comparison grid: {e}")

    print(f"    ✓ Graphs created successfully")
    sys.stdout.flush()

def create_parameter_sweep_graphs(all_results, graph_dir, noise_folder_name, best_param):
    """Create line graphs showing metrics vs parameters"""
    print(f"    Creating parameter sweep graphs...")
    sys.stdout.flush()

    metrics = ["PSNR", "SSIM", "MSE", "Noise_Variance", "Entropy_Diff", "Sharpness"]

    param_values = []
    param_names = []

    for param_name in all_results.keys():
        param_names.append(param_name)
        
        try:
            if 'kmedoids_k' in param_name:
                parts = param_name.split('_')
                for part in parts:
                    if part.startswith('k') and part[1:].isdigit():
                        num = int(part[1:])
                        break
                else:
                    num = 0
            elif 'nlm_' in param_name:
                parts = param_name.split('_')
                num = 0.0
                for part in parts:
                    if part.startswith('h'):
                        try:
                            num = float(part[1:])
                        except ValueError:
                            num = 0.0
                        break
            else:
                num = 0
        except Exception:
            num = 0
        
        param_values.append(num)

    sorted_indices = np.argsort(param_values)
    sorted_params = [param_names[i] for i in sorted_indices]
    sorted_values = [param_values[i] for i in sorted_indices]

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()

    for idx, metric in enumerate(metrics):
        ax = axes[idx]

        means = []
        for param_name in sorted_params:
            df = all_results[param_name]
            means.append(df[metric].mean())

        ax.plot(sorted_values, means, marker='o', linewidth=2, markersize=8, color='steelblue')

        best_idx = sorted_params.index(best_param)
        ax.axvline(x=sorted_values[best_idx], color='red', linestyle='--', linewidth=2,
                   label=f'Best: {sorted_values[best_idx]}')
        ax.plot(sorted_values[best_idx], means[best_idx], 'ro', markersize=12)

        if 'kmedoids' in sorted_params[0]:
            param_type = "Clusters (k)"
        elif 'nlm_' in sorted_params[0]:
            param_type = "Filter strength h"
        else:
            param_type = "Parameter Value"
        
        ax.set_xlabel(f'{param_type}', fontsize=12, fontweight='bold')
        ax.set_ylabel(metric, fontsize=12, fontweight='bold')
        ax.set_title(f'{metric} vs {param_type}', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()

    plt.suptitle(f'Parameter Sweep Analysis - {noise_folder_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(graph_dir, 'parameter_sweep.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"    ✓ Parameter sweep graphs created")
    sys.stdout.flush()

def create_visual_comparison_grid(noise_folder_path, original_images_path, all_results,
                                   graph_dir, noise_folder_name, best_param,
                                   is_salt_pepper, is_speckle):
    """Create a grid showing visual comparison of denoising with different parameters"""
    print(f"    Creating visual comparison grid...")
    sys.stdout.flush()

    noisy_files = [f for f in os.listdir(noise_folder_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not noisy_files:
        return

    sample_file = noisy_files[0]
    noisy_path = os.path.join(noise_folder_path, sample_file)
    noisy_img = cv2.imread(noisy_path)

    if sample_file.lower().startswith("noisy_"):
        original_filename = sample_file[6:]
    else:
        original_filename = sample_file

    original_path = os.path.join(original_images_path, original_filename)
    original_img = cv2.imread(original_path)

    if original_img is None or noisy_img is None:
        return

    noisy_img = _resize_to_match(original_img, noisy_img)

    param_list = sorted(all_results.keys())
    total_images = 2 + len(param_list)
    cols = 5
    rows = (total_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(20, 4*rows))
    axes = axes.flatten()

    axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Noisy', fontsize=12, fontweight='bold', color='red')
    axes[1].axis('off')

    for idx, param_name in enumerate(param_list):
        ax = axes[idx + 2]

        try:
            if is_salt_pepper:
                params = next(p for p in SP_DENOISE_PARAMS['kmedoids'] if p['name'] == param_name)
                denoised_img = denoise_salt_pepper_kmedoids(
                    noisy_img,
                    n_clusters=params['n_clusters'],
                    use_median_prefilter=params['use_median_prefilter']
                )
            elif is_speckle:
                params = next(p for p in SPECKLE_DENOISE_PARAMS['nlm'] if p['name'] == param_name)
                denoised_img = denoise_speckle_nlm(
                    noisy_img,
                    h=params["h"],
                    templateWindowSize=params["template"],
                    searchWindowSize=params["search"]
                )

            denoised_img = _resize_to_match(original_img, denoised_img)
            ax.imshow(cv2.cvtColor(denoised_img, cv2.COLOR_BGR2RGB))

            df = all_results[param_name]
            avg_psnr = df['PSNR'].mean()

            if param_name == best_param:
                title_color = 'green'
                title = f'{param_name}\nPSNR: {avg_psnr:.2f} ⭐ BEST'
            else:
                title_color = 'black'
                title = f'{param_name}\nPSNR: {avg_psnr:.2f}'

            ax.set_title(title, fontsize=10, fontweight='bold', color=title_color)
            ax.axis('off')

        except Exception as e:
            ax.text(0.5, 0.5, f'Error:\n{str(e)[:50]}', ha='center', va='center')
            ax.set_title(param_name, fontsize=10)
            ax.axis('off')

    for idx in range(total_images, len(axes)):
        axes[idx].axis('off')

    plt.suptitle(f'Visual Comparison - {noise_folder_name}\nSample: {sample_file}',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(graph_dir, 'visual_comparison_grid.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"    ✓ Visual comparison grid created")
    sys.stdout.flush()

def create_best_comparison_image(noise_folder_path, original_images_path, all_results,
                                  graph_dir, noise_folder_name, best_param,
                                  is_salt_pepper, is_speckle):
    """Create side-by-side comparison of original, noisy, and best denoised"""
    print(f"      Creating best parameter comparison...")
    sys.stdout.flush()

    try:
        noisy_files = [f for f in os.listdir(noise_folder_path)
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not noisy_files:
            return

        sample_file = noisy_files[0]
        noisy_path = os.path.join(noise_folder_path, sample_file)
        noisy_img = cv2.imread(noisy_path)

        if noisy_img is None:
            return

        if sample_file.lower().startswith("noisy_"):
            original_filename = sample_file[6:]
        else:
            original_filename = sample_file

        original_path = os.path.join(original_images_path, original_filename)
        original_img = cv2.imread(original_path)

        if original_img is None:
            return

        noisy_img = _resize_to_match(original_img, noisy_img)

        if is_salt_pepper:
            params = next(p for p in SP_DENOISE_PARAMS['kmedoids'] if p['name'] == best_param)
            denoised_img = denoise_salt_pepper_kmedoids(
                noisy_img,
                n_clusters=params['n_clusters'],
                use_median_prefilter=params['use_median_prefilter']
            )
        elif is_speckle:
            params = next(p for p in SPECKLE_DENOISE_PARAMS['nlm'] if p['name'] == best_param)
            denoised_img = denoise_speckle_nlm(
                noisy_img,
                h=params["h"],
                templateWindowSize=params["template"],
                searchWindowSize=params["search"]
            )

        denoised_img = _resize_to_match(original_img, denoised_img)

        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        axes[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0].axis('off')

        axes[1].imshow(cv2.cvtColor(noisy_img, cv2.COLOR_BGR2RGB))
        axes[1].set_title('Noisy Image', fontsize=14, fontweight='bold', color='red')
        axes[1].axis('off')

        axes[2].imshow(cv2.cvtColor(denoised_img, cv2.COLOR_BGR2RGB))

        df = all_results[best_param]
        avg_psnr = df['PSNR'].mean()
        avg_ssim = df['SSIM'].mean()

        axes[2].set_title(
            f'Denoised ({best_param})\nPSNR: {avg_psnr:.2f} dB | SSIM: {avg_ssim:.4f}',
            fontsize=14, fontweight='bold', color='green'
        )
        axes[2].axis('off')

        plt.suptitle(f'Best Denoising Result - {noise_folder_name}', fontsize=16, fontweight='bold')
        plt.tight_layout()

        output_path = os.path.join(graph_dir, 'best_comparison.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()

        print(f"        ✓ Successfully saved best_comparison.png")
        sys.stdout.flush()

    except Exception as e:
        print(f"        ERROR creating best comparison: {e}")

# ======================== MAIN DENOISING PIPELINE ========================
def process_noise_folder(bw_folder_path, colour_folder_path, noise_folder_name, 
                        original_images_path, output_base, is_salt_pepper, is_speckle):
    """Process both BW and colour images for a single noise parameter."""
    
    sys.stdout.flush()
    
    # Create output folders
    output_dir = os.path.join(output_base, noise_folder_name)
    graph_dir = os.path.join(output_dir, "Graph_data")
    csv_dir = os.path.join(output_dir, "Denoised_csv")
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(graph_dir, exist_ok=True)
    os.makedirs(csv_dir, exist_ok=True)
    
    if is_salt_pepper:
        denoise_methods = SP_DENOISE_PARAMS
        print(f"Noise Type: Salt & Pepper - {noise_folder_name}")
    elif is_speckle:
        denoise_methods = SPECKLE_DENOISE_PARAMS
        print(f"Noise Type: Speckle - {noise_folder_name}")
    else:
        print(f"Unknown noise type for {noise_folder_name}")
        return None, None
    
    # Collect all noisy images from both BW and colour folders
    noisy_files_bw = []
    noisy_files_colour = []
    
    if os.path.exists(bw_folder_path):
        noisy_files_bw = [f for f in os.listdir(bw_folder_path) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if os.path.exists(colour_folder_path):
        noisy_files_colour = [f for f in os.listdir(colour_folder_path)
                             if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    total_files = len(noisy_files_bw) + len(noisy_files_colour)
    print(f"Found {len(noisy_files_bw)} BW + {len(noisy_files_colour)} Colour = {total_files} images")
    
    all_results = {}
    
    for method_name, params_list in denoise_methods.items():
        for params in params_list:
            param_name = params["name"]
            print(f"  Testing: {param_name}")
            sys.stdout.flush()
            
            metrics_list = []
            
            # Process BW images
            for noisy_filename in tqdm(noisy_files_bw, desc=f"    BW {param_name}"):
                noisy_path = os.path.join(bw_folder_path, noisy_filename)
                noisy_img = cv2.imread(noisy_path)
                if noisy_img is None:
                    continue
                
                if noisy_filename.lower().startswith("noisy_"):
                    original_filename = noisy_filename[6:]
                else:
                    original_filename = noisy_filename
                
                original_path = os.path.join(original_images_path, original_filename)
                original_img = cv2.imread(original_path)
                if original_img is None:
                    continue
                
                noisy_img = _resize_to_match(original_img, noisy_img)
                
                try:
                    if is_salt_pepper:
                        denoised_img = denoise_salt_pepper_kmedoids(
                            noisy_img, 
                            n_clusters=params["n_clusters"],
                            use_median_prefilter=params["use_median_prefilter"]
                        )
                    elif is_speckle:
                        denoised_img = denoise_speckle_nlm(
                            noisy_img,
                            h=params["h"],
                            templateWindowSize=params["template"],
                            searchWindowSize=params["search"]
                        )
                    
                    denoised_img = _resize_to_match(original_img, denoised_img)
                    metrics = calculate_all_metrics(original_img, denoised_img, noisy_img)
                    metrics["Name"] = f"BW_{noisy_filename}"
                    metrics_list.append(metrics)
                except Exception as e:
                    print(f"      Error processing BW {noisy_filename}: {e}")
                    continue
            
            # Process Colour images
            for noisy_filename in tqdm(noisy_files_colour, desc=f"    Colour {param_name}"):
                noisy_path = os.path.join(colour_folder_path, noisy_filename)
                noisy_img = cv2.imread(noisy_path)
                if noisy_img is None:
                    continue
                
                if noisy_filename.lower().startswith("noisy_"):
                    original_filename = noisy_filename[6:]
                else:
                    original_filename = noisy_filename
                
                original_path = os.path.join(original_images_path, original_filename)
                original_img = cv2.imread(original_path)
                if original_img is None:
                    continue
                
                noisy_img = _resize_to_match(original_img, noisy_img)
                
                try:
                    if is_salt_pepper:
                        denoised_img = denoise_salt_pepper_kmedoids(
                            noisy_img,
                            n_clusters=params["n_clusters"],
                            use_median_prefilter=params["use_median_prefilter"]
                        )
                    elif is_speckle:
                        denoised_img = denoise_speckle_nlm(
                            noisy_img,
                            h=params["h"],
                            templateWindowSize=params["template"],
                            searchWindowSize=params["search"]
                        )
                    
                    denoised_img = _resize_to_match(original_img, denoised_img)
                    metrics = calculate_all_metrics(original_img, denoised_img, noisy_img)
                    metrics["Name"] = f"Colour_{noisy_filename}"
                    metrics_list.append(metrics)
                except Exception as e:
                    print(f"      Error processing Colour {noisy_filename}: {e}")
                    continue
            
            if metrics_list:
                df = pd.DataFrame(metrics_list)
                column_order = ["Name", "PSNR", "SSIM", "MSE", "Entropy_Diff",
                               "Noise_Variance", "Sharpness", "Spatial_Freq", "Dynamic_Range"]
                df = df[column_order]
                
                csv_filename = f"Denoised_{param_name}.csv"
                csv_path = os.path.join(csv_dir, csv_filename)
                df.to_csv(csv_path, index=False)
                print(f"      Saved CSV: {csv_filename}")
                sys.stdout.flush()
                
                all_results[param_name] = df
    
    if not all_results:
        print(f"  ERROR: No results generated for {noise_folder_name}")
        return None, None
    
    print(f"\n  Finding best denoising parameters...")
    sys.stdout.flush()
    
    best_param_name, best_psnr, best_ssim = select_best_parameter_psnr_detail(all_results, psnr_tolerance=0.5)
    
    avg_metrics = {}
    for param_name, df in all_results.items():
        avg_metrics[param_name] = {
            "PSNR": df["PSNR"].mean(),
            "SSIM": df["SSIM"].mean(),
            "MSE": df["MSE"].mean()
        }
    
    best_param = (best_param_name, avg_metrics[best_param_name])
    print(f"    Best parameters: {best_param[0]}")
    print(f"    Average PSNR: {best_param[1]['PSNR']:.2f}")
    print(f"    Average SSIM: {best_param[1]['SSIM']:.4f}")
    sys.stdout.flush()
    
    # Create visualizations using BW folder as primary
    print(f"\n  Creating visualizations...")
    sys.stdout.flush()
    
    try:
        create_comparison_graphs(all_results, graph_dir, noise_folder_name, best_param[0])
        create_parameter_sweep_graphs(all_results, graph_dir, noise_folder_name, best_param[0])
        
        # Use BW folder for visual comparisons
        if os.path.exists(bw_folder_path) and noisy_files_bw:
            create_visual_comparison_grid(
                bw_folder_path, original_images_path, all_results,
                graph_dir, noise_folder_name, best_param[0],
                is_salt_pepper, is_speckle
            )
            create_best_comparison_image(
                bw_folder_path, original_images_path, all_results,
                graph_dir, noise_folder_name, best_param[0],
                is_salt_pepper, is_speckle
            )
        
        print(f"    ✓ All visualizations created successfully")
        sys.stdout.flush()
        
    except Exception as e:
        print(f"    ERROR creating graphs: {e}")
        import traceback
        traceback.print_exc()
    
    return best_param[0], avg_metrics

# ======================== MAIN EXECUTION ========================
def main():
    
    sys.stdout.flush()
    
    # Debug: Print current working directory
    import os
    print(f"\nDEBUG: Current working directory: {os.getcwd()}")
    print(f"DEBUG: Checking if paths exist...")
    print(f"  BASE_PATH = {BASE_PATH}")
    print(f"  os.path.exists(BASE_PATH) = {os.path.exists(BASE_PATH)}")
    print(f"  os.path.abspath(BASE_PATH) = {os.path.abspath(BASE_PATH)}")
    
    print(f"\n  BW_INPUT = {BW_INPUT}")
    print(f"  os.path.exists(BW_INPUT) = {os.path.exists(BW_INPUT)}")
    print(f"  os.path.abspath(BW_INPUT) = {os.path.abspath(BW_INPUT)}")
    
    print(f"\n  COLOUR_INPUT = {COLOUR_INPUT}")
    print(f"  os.path.exists(COLOUR_INPUT) = {os.path.exists(COLOUR_INPUT)}")
    print(f"  os.path.abspath(COLOUR_INPUT) = {os.path.abspath(COLOUR_INPUT)}")
    
    original_images_path = BASE_PATH
    
    # Check if paths exist
    paths_ok = True
    if not os.path.exists(BASE_PATH):
        print(f"\n  ERROR: Base path does not exist: {BASE_PATH}")
        paths_ok = False
    
    if not os.path.exists(BW_INPUT):
        print(f" ERROR: BW input path does not exist: {BW_INPUT}")
        paths_ok = False
    
    if not os.path.exists(COLOUR_INPUT):
        print(f" ERROR: Colour input path does not exist: {COLOUR_INPUT}")
        paths_ok = False
    
    if not paths_ok:
        print("\n  Please check your folder paths and try again.")
        print("\nDEBUG: Listing current directory contents:")
        try:
            contents = os.listdir(".")
            print(f"  Contents of '.': {contents}")
            if "Photos_Subset" in contents:
                print(f"  Contents of './Photos_Subset': {os.listdir('./Photos_Subset')}")
        except Exception as e:
            print(f"  Error listing directory: {e}")
        return
    
    best_parameters = {}
    
    # Rest of the code remains the same...
    # Get all folders from BW and Colour
    bw_folders = set()
    colour_folders = set()
    
    # List directories in BW folder
    if os.path.exists(BW_INPUT) and os.path.isdir(BW_INPUT):
        for f in os.listdir(BW_INPUT):
            full_path = os.path.join(BW_INPUT, f)
            if os.path.isdir(full_path):
                bw_folders.add(f)
    
    # List directories in Colour folder  
    if os.path.exists(COLOUR_INPUT) and os.path.isdir(COLOUR_INPUT):
        for f in os.listdir(COLOUR_INPUT):
            full_path = os.path.join(COLOUR_INPUT, f)
            if os.path.isdir(full_path):
                colour_folders.add(f)
    
    # Combine and separate into salt_pepper and speckle
    all_folders = bw_folders.union(colour_folders)
    sp_folders = sorted([f for f in all_folders if f.startswith("amount_")])
    speckle_folders = sorted([f for f in all_folders if f.startswith("intensity_")])
    
    # Process Salt & Pepper

    print(f"Found {len(sp_folders)} salt & pepper parameter folders: {sp_folders}")
    
    for idx, folder in enumerate(sp_folders, 1):
        print(f"\n[{idx}/{len(sp_folders)}] Processing {folder}")
        bw_path = os.path.join(BW_INPUT, folder)
        colour_path = os.path.join(COLOUR_INPUT, folder)
        
        try:
            best_param, avg_metrics = process_noise_folder(
                bw_path, colour_path, folder,
                original_images_path, OUTPUT_SP,
                is_salt_pepper=True, is_speckle=False
            )
            if best_param and avg_metrics:
                best_parameters[("salt_pepper", folder)] = {
                    "best_param": best_param,
                    "metrics": avg_metrics
                }
        except Exception as e:
            print(f"Error processing {folder}: {e}")
            import traceback
            traceback.print_exc()
    
    # Process Speckle
    
    print(f"Found {len(speckle_folders)} speckle parameter folders: {speckle_folders}")
    
    for idx, folder in enumerate(speckle_folders, 1):
        print(f"\n[{idx}/{len(speckle_folders)}] Processing {folder}")
        bw_path = os.path.join(BW_INPUT, folder)
        colour_path = os.path.join(COLOUR_INPUT, folder)
        
        try:
            best_param, avg_metrics = process_noise_folder(
                bw_path, colour_path, folder,
                original_images_path, OUTPUT_SPECKLE,
                is_salt_pepper=False, is_speckle=True
            )
            if best_param and avg_metrics:
                best_parameters[("speckle", folder)] = {
                    "best_param": best_param,
                    "metrics": avg_metrics
                }
        except Exception as e:
            print(f"Error processing {folder}: {e}")
            import traceback
            traceback.print_exc()
    
    # Summary
    
    for (noise_type, folder), result in best_parameters.items():
        print(f"\n[{noise_type}] {folder}:")
        print(f"  Best Method: {result['best_param']}")
        print(f"  Avg PSNR: {result['metrics'][result['best_param']]['PSNR']:.2f}")
        print(f"  Avg SSIM: {result['metrics'][result['best_param']]['SSIM']:.4f}")
    
    print(f"\n✓ Results saved to:")
    print(f"  Salt & Pepper: {OUTPUT_SP}")
    print(f"  Speckle: {OUTPUT_SPECKLE}")

if __name__ == "__main__":
    main()

Cleaning existing output directory: ../../Data_Results/Data/finished/salt_pepper
Cleaning existing output directory: ../../Data_Results/Data/finished/speckle
✓ Base (clean) images: ../../Photos_Subset/Original
✓ BW noised images: ../../Photos_Subset/Noised_Images/BW
✓ Colour noised images: ../../Photos_Subset/Noised_Images/Color
✓ Salt & Pepper output: ../../Data_Results/Data/finished/salt_pepper
✓ Speckle output: ../../Data_Results/Data/finished/speckle


DEBUG: Current working directory: /Users/varadarohokale/Desktop/CS 506 Project/Noise_Reduction_CS506/photo_editing/salt_pepper_and_speckle
DEBUG: Checking if paths exist...
  BASE_PATH = ../../Photos_Subset/Original
  os.path.exists(BASE_PATH) = True
  os.path.abspath(BASE_PATH) = /Users/varadarohokale/Desktop/CS 506 Project/Noise_Reduction_CS506/Photos_Subset/Original

  BW_INPUT = ../../Photos_Subset/Noised_Images/BW
  os.path.exists(BW_INPUT) = True
  os.path.abspath(BW_INPUT) = /Users/varadarohokale/Desktop/CS 506 Project/Noise_R

    Colour kmedoids_k3_no_prefilter: 100%|██████████| 80/80 [00:02<00:00, 27.89it/s]

      Saved CSV: Denoised_kmedoids_k3_no_prefilter.csv
  Testing: kmedoids_k5_no_prefilter



    Colour kmedoids_k5_no_prefilter: 100%|██████████| 80/80 [00:03<00:00, 23.26it/s]

      Saved CSV: Denoised_kmedoids_k5_no_prefilter.csv
  Testing: kmedoids_k8_no_prefilter


...


    Colour nlm_h23_t9_s31: 100%|██████████| 80/80 [00:06<00:00, 12.09it/s]

      Saved CSV: Denoised_nlm_h23_t9_s31.csv

  Finding best denoising parameters...
    Best parameters: nlm_h20_t7_s21
    Average PSNR: 14.40
    Average SSIM: 0.5145

  Creating visualizations...
    Creating graphs for intensity_3.0...


    ✓ Graphs created successfully
    Creating parameter sweep graphs...
    ✓ Parameter sweep graphs created
    Creating visual comparison grid...


/var/folders/yk/j6fv9zxs1cj6xyv3yj1wfd7h0000gn/T/ipykernel_14681/2006138128.py:637: UserWarning: Glyph 11088 (\N{WHITE MEDIUM STAR}) missing from font(s) Arial.
  plt.tight_layout()
/var/folders/yk/j6fv9zxs1cj6xyv3yj1wfd7h0000gn/T/ipykernel_14681/2006138128.py:638: UserWarning: Glyph 11088 (\N{WHITE MEDIUM STAR}) missing from font(s) Arial.
  plt.savefig(os.path.join(graph_dir, 'visual_comparison_grid.png'), dpi=300, bbox_inches='tight')


    ✓ Visual comparison grid created
      Creating best parameter comparison...
        ✓ Successfully saved best_comparison.png
    ✓ All visualizations created successfully

[salt_pepper] amount_0.01:
  Best Method: kmedoids_k12_median
  Avg PSNR: 28.80
  Avg SSIM: 0.9157

[salt_pepper] amount_0.05:
  Best Method: kmedoids_k12_median
  Avg PSNR: 27.65
  Avg SSIM: 0.8935

[salt_pepper] amount_0.10:
  Best Method: kmedoids_k10_median
  Avg PSNR: 26.16
  Avg SSIM: 0.8662

[salt_pepper] amount_0.15:
  Best Method: kmedoids_k10_median
  Avg PSNR: 24.88
  Avg SSIM: 0.8416

[salt_pepper] amount_0.20:
  Best Method: kmedoids_k8_median
  Avg PSNR: 23.06
  Avg SSIM: 0.7946

[speckle] intensity_0.5:
  Best Method: nlm_h14_t7_s21
  Avg PSNR: 21.77
  Avg SSIM: 0.7856

[speckle] intensity_1.0:
  Best Method: nlm_h17_t7_s21
  Avg PSNR: 17.46
  Avg SSIM: 0.6541

[speckle] intensity_1.5:
  Best Method: nlm_h17_t7_s21
  Avg PSNR: 15.71
  Avg SSIM: 0.5620

[speckle] intensity_2.0:
  Best Method: nlm_h